In [127]:
import pandas as pd 
import numpy as np

In [128]:
#Read csv
# Create a path for the CSV file desired
csv_students = "./raw_data/students_complete.csv"

# Read the CSV into a Pandas DataFrame
students_df = pd.read_csv(csv_students)

In [129]:
#Read csv
# Create a reference the CSV file desired
csv_schools = "./raw_data/schools_complete.csv"

# Read the CSV into a Pandas DataFrame
schools_df = pd.read_csv(csv_schools)

In [134]:
#Rename columns in data frames to be more descriptive
schools_df_renamed = schools_df.rename(columns ={"name": "School Name",
                                                 "type": "School Type",
                                                 "size": "School Size",
                                                 "budget": "School Budget"})
students_df_renamed = students_df.rename(columns={"name": "Student Name",
                                                  "gender":"Student Gender",
                                                  "grade": "Student Grade",
                                                  "school": "School Name",
                                                  "reading_score": "Student Reading Score",
                                                  "math_score": "Student Math Score"})


In [169]:
# Find number of schools
unique_schools = len(schools_df_renamed["School Name"])
#Find number of students
num_students = len(students_df_renamed["Student Name"])
#Find total schools' budget
total_budget = schools_df_renamed["School Budget"].sum()
#Find average math score
avg_math = students_df_renamed["Student Math Score"].mean()
#Find average reading score
avg_read = students_df_renamed["Student Reading Score"].mean()   
#Find percent passed math
student_math = students_df_renamed[students_df_renamed["Student Math Score"] >= 70]
stu_passed_math = (len(student_math) / len(students_df_renamed)) * 100
#Find percent passed reading
student_reading = students_df_renamed[students_df_renamed["Student Reading Score"] >= 70]
stu_passed_reading = (len(student_reading) / len(students_df_renamed)) * 100   
#Find percent passed overall
overall_pass = (stu_passed_reading + stu_passed_math) / 2

In [170]:
#***Create District Summary Table 
district_summary = pd.DataFrame(
    {"Total Schools": [unique_schools],
     "Total Students": [num_students],
     "Total Budget": [total_budget],
     "Average Math Score": [avg_math],
     "Average Reading Score": [avg_read],
     "% Passing Math": [stu_passed_math],
     "% Passing Reading": [stu_passed_reading],
     "% Overall Passing Rate": [overall_pass]})

column_names = ["Total Schools", "Total Students","Total Budget", "Average Math Score", 
                "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]

district_summary = district_summary[column_names]
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


In [218]:
#Merge schools_df_renamed and students_df_renamed
merge_table = pd.merge(schools_df_renamed, students_df_renamed, on="School Name")

In [219]:
#Find average reading score for each school
avg_reading_score = merge_table.groupby(["School Name"])["Student Reading Score"].mean()
avg_reading_score = pd.DataFrame(avg_reading_score)
avg_reading_score = avg_reading_score.rename(columns ={"Student Reading Score": "Average Student Reading Score"})
avg_reading_score = avg_reading_score.reset_index()

#Find Average Math Score by School
avg_math_score = merge_table.groupby(["School Name"])["Student Math Score"].mean()
avg_math_score = pd.DataFrame(avg_math_score)
avg_math_score = avg_math_score.rename(columns ={"Student Math Score": "Average Student Math Score"})
avg_math_score = avg_math_score.reset_index().head()


#Find percent passing math
#Define function name and variable names
def passing_grade(df, column, school_name):
    #no_passing = the number of items in the the identified merge_table column if they are >= 70 and are part of the unique school name in the for loop
    no_passing = len(merge_table[(df[column] >= 70) & (merge_table['School Name'] == school_name)])
    #total_students = the number of rows in which that school name appears
    total_students = len(merge_table[merge_table['School Name'] == school_name])
    #pct_passing = the number of students who passed divided by the total number of students
    pct_passing = (no_passing/total_students) * 100
    #return a dictionary containing the school name: and the percent passed
    return {school_name:pct_passing} 

#Create new dictionary to hold lists of school names and percent passed per school
passed_math_dict = {"School Name":[],"Percent Passed Math": []}
#loop through the unique values in the school name column in merge_table
for school_name in merge_table['School Name'].unique():
    #Run the passing_grade function on the Student Math Score column of the merge_table
    temp_return = passing_grade(merge_table, 'Student Math Score', school_name)
    #Loop through temp_return
    for key in temp_return:
        #append each school name to the school name list in the dictionary
        passed_math_dict["School Name"].append(key)
        #append each percentage to the "Percent Math Passed" list in the dictionary
        passed_math_dict["Percent Passed Math"].append(temp_return[key])
#Put the columns in order
columns_math = ["School Name", "Percent Passed Math"]
#Create dataframe
passed_math_school = pd.DataFrame(passed_math_dict)[columns_math]
passed_math_school.head()

#Find percent passing reading
#Define function name and variable names
def passing_grade(df, column, school_name):
    #no_passing = the number of items in the the identified merge_table column if they are >= 70 and are part of the unique school name in the for loop
    no_passing = len(merge_table[(df[column] >= 70) & (merge_table['School Name'] == school_name)])
    #total_students = the number of rows in which that school name appears
    total_students = len(merge_table[merge_table['School Name'] == school_name])
    #pct_passing = the number of students who passed divided by the total number of students
    pct_passing = (no_passing/total_students) * 100
    #return a dictionary containing the school name: and the percent passed
    return {school_name:pct_passing} 

#Create new dictionary to hold lists of school names and percent passed per school
passed_reading_dict = {"School Name":[],"Percent Passed Reading": []}
#loop through the unique values in the school name column in merge_table
for school_name in merge_table['School Name'].unique():
    #Run the passing_grade function on the Student Reading Score column of the merge_table
    temp_return = passing_grade(merge_table, 'Student Reading Score', school_name)
    #Loop through temp_return
    for key in temp_return:
        #append each school name to the school name list in the dictionary
        passed_reading_dict["School Name"].append(key)
        #append each percentage to the "Percent Passed Reading" list in the dictionary
        passed_reading_dict["Percent Passed Reading"].append(temp_return[key])
#Put the columns in order
columns_reading = ["School Name", "Percent Passed Reading"]
#Create dataframe
passed_reading_school = pd.DataFrame(passed_reading_dict)[columns_reading]

In [ ]:
#***Top 5 Performing Schools: sort School Summary Table by descending "% Overall Passing Rate" and show only .head()

In [ ]:
#***Bottom 5 Performing Schools: sort School Summary Table by ascending "% Overall Passing Rate" and show only .head()

In [120]:
#***Create table that lists the average math score for students of each grade level
math_by_grade = merge_table.drop(["School ID", "School Type", "School Size", "School Budget", 
                                  "Student ID", "Student Name", "Student Gender", "Average Student Reading Score", 
                                  "Mean Student Math Score", "Student Reading Score"], axis=1)
math_by_grade = math_by_grade.rename(columns ={"Student Math Score": "Average Student Math Score"})
math_by_grade.groupby(["School Name", "Student Grade"]).mean().head()

Average Student Math Score
School Name         Student Grade                            
Bailey High School  10th                            76.996772
                    11th                            77.515588
                    12th                            76.492218
                    9th                             77.083676
Cabrera High School 10th                            83.154506

In [121]:
#***Create table that lists the average reading score for students of each grade level
reading_by_grade = merge_table.drop(["School ID", "School Type", "School Size", "School Budget", 
                                  "Student ID", "Student Name", "Student Gender", "Average Student Reading Score", 
                                  "Mean Student Math Score", "Student Math Score"], axis=1)
reading_by_grade = reading_by_grade.rename(columns ={"Student Reading Score": "Average Student Reading Score"})
reading_by_grade.groupby(["School Name", "Student Grade"]).mean().head()

Average Student Reading Score
School Name         Student Grade                               
Bailey High School  10th                               80.907183
                    11th                               80.945643
                    12th                               80.912451
                    9th                                81.303155
Cabrera High School 10th                               84.253219

In [ ]:
#***Create